In [ ]:
import pathlib
import tempfile
from urllib.parse import urlparse
from urllib.request import urlretrieve

import numpy as np
import asdf
from astropy.io import fits
from astropy.modeling import models, fitting
import astropy.units as u
import matplotlib.pyplot as plt

from gwcs import wcs as gwcs
from gwcs import coordinate_frames as cf

from grism_observation import GrismObs
from HST.hst_grism_reffiles import create_tsgrism_wavelengthrange, create_grism_specwcs
from HST.generate_wfc3_distortion import create_wfc3_distortion

In [ ]:
# Try out a GrismObs for UVIS

sip_file = "/Users/rosteen/Data/UVIS/G280/icwz15e7q_flt.fits"

test = GrismObs(sip_file)

print(test.geometric_transforms["CCD1"].available_frames)

g2d = test.geometric_transforms["CCD1"].get_transform("grism_detector", "detector")
d2g = test.geometric_transforms["CCD1"].get_transform("detector", "grism_detector")

In [ ]:
d2g(1500.0, 1500.0, 5000, 1.0)

In [ ]:
g2d(1215.5599807197782, 1681.6227528517936, 1500.0, 1500.0, 1.0)

In [ ]:
g2w = test.geometric_transforms["CCD1"].get_transform("grism_detector", "world")
w2g = test.geometric_transforms["CCD1"].get_transform("world", "grism_detector")

In [ ]:
w2g(206.4318333333, 26.41859444444, 3000, 1.0)

In [ ]:
# Well this sure looks horrible.
g2w(1941.4204046114864,
 1245.8273935887883,
 2047.2667314355476,
 1070.8144590508862,
 1.0)

In [ ]:
d2w = test.geometric_transforms["CCD1"].get_transform("detector", "world")
w2d = test.geometric_transforms["CCD1"].get_transform("world", "detector")

In [ ]:
test.grism_header

In [ ]:
w2d(206.4318333333, 26.41859444444, 6000, 1.0)

In [ ]:
d2w(2047.2667314355476, 1070.8144590508862, 6000.0, 1.0)

#### To compare with GRISMCONF

In [ ]:
from grismconf import Config

C = Config("/Users/rosteen/projects/GRISM_WFC3/UVIS/UVIS_G280_CCD1_V2.conf")

In [ ]:
w0,w1 = C.WRANGE["+1"] # wav range of order
print(w0, w1)

In [ ]:
t0,t1 = C.INVDISPL("+1",1500,1500,[w0,w1]) #  validd ts  values for order
print(t0, t1)

In [ ]:
xs = [C.DISPX("+1",1500,1500,x) for x in np.arange(t0,t1,0.01)]
lam = [C.DISPL("+1",1500,1500,x) for x in np.arange(t0,t1,0.01)]
plt.plot(xs,lam)

In [ ]:
import asdf
from HST.dispersion_models import DISPXY_Extension

asdf.get_config().add_extension(DISPXY_Extension())

specwcs = asdf.open("config/HST/WFC3_G280_CCD1_specwcs.asdf").tree

lmodel = specwcs['displ'][0]

l = lmodel.evaluate(1500, 1500, np.arange(t0,t1,0.01))
plt.plot(xs, l)